## TO DOs (internal)
* add dual values to results

### Reminder (Before we start)
Change the kernel at the top and set it to "urbs" to be able to run this script.

# Example 1: Electricity supply of an island

## Learning objectives
* Translate a mathematical optimization problem into a pyomo ConcreteModel/AbstractModel
* Recognize the basic structure of an optimization model
* Report the results of pyomo in different formats

## Mathematical formulation

We start with a simple example. Let's assume we have a gas power plant ($P_{gas}$ = 100 MW) and a biomass power plant ($P_{bm}$ = 30 MW) supplying an island. The cost of supplying 1 MWh of electricity using the gas power plant is EUR 50, whereas the cost of using biomass is 25 EUR/MWh.
We would like to minimize the cost of operating the system for a given demand of electricity $d(t)$.

$$\min \quad 50s_{gas}(t) + 25s_{bm}(t)$$
$$s.t. \quad s_{gas}(t) + s_{bm}(t) \geq d(t)$$

The supply from the power plant is non-negative:
$$s_{gas}(t), s_{bm}(t) \geq 0$$

It cannot exceed the capacity of the power plants:
$$s_{gas}(t) \leq 100$$
$$s_{bm}(t) \leq 30$$

Further, we define the demand as follows:
$$d(t) = [60, 100, 120, 80, 30]$$

***
### <span style="color:blue">Task</span>
Try to solve this problem with pen and paper!
***

## Formulation as a pyomo ConcreteModel

We could solve this problem using a pyomo ConcreteModel:

In [1]:
import pyomo.environ as pyo
from pyomo.environ import *

model = pyo.ConcreteModel()
model.name = "Example1"

# Variables
model.s = pyo.Var(RangeSet(1, 5), RangeSet(1, 2), domain=pyo.NonNegativeReals)

# Objective function
model.OBJ = pyo.Objective(expr=50*model.s[1,1] + 25*model.s[1,2] +\
                               50*model.s[2,1] + 25*model.s[2,2] +\
                               50*model.s[3,1] + 25*model.s[3,2] +\
                               50*model.s[4,1] + 25*model.s[4,2] +\
                               50*model.s[5,1] + 25*model.s[5,2])

# Constraints
model.ConstraintGasCap1 = pyo.Constraint(expr = model.s[1, 1] <= 100)
model.ConstraintGasCap2 = pyo.Constraint(expr = model.s[2, 1] <= 100)
model.ConstraintGasCap3 = pyo.Constraint(expr = model.s[3, 1] <= 100)
model.ConstraintGasCap4 = pyo.Constraint(expr = model.s[4, 1] <= 100)
model.ConstraintGasCap5 = pyo.Constraint(expr = model.s[5, 1] <= 100)

model.ConstraintBiomassCap1 = pyo.Constraint(expr = model.s[1, 2] <= 30)
model.ConstraintBiomassCap2 = pyo.Constraint(expr = model.s[2, 2] <= 30)
model.ConstraintBiomassCap3 = pyo.Constraint(expr = model.s[3, 2] <= 30)
model.ConstraintBiomassCap4 = pyo.Constraint(expr = model.s[4, 2] <= 30)
model.ConstraintBiomassCap5 = pyo.Constraint(expr = model.s[5, 2] <= 30)

model.ConstraintDem1 = pyo.Constraint(expr = model.s[1,1] + model.s[1,2] >= 60)
model.ConstraintDem2 = pyo.Constraint(expr = model.s[2,1] + model.s[2,2] >= 100)
model.ConstraintDem3 = pyo.Constraint(expr = model.s[3,1] + model.s[3,2] >= 120)
model.ConstraintDem4 = pyo.Constraint(expr = model.s[4,1] + model.s[4,2] >= 80)
model.ConstraintDem5 = pyo.Constraint(expr = model.s[5,1] + model.s[5,2] >= 30)

In [3]:
# Write the LP mathematical problem that is solved to a file (optional)
# Here, we are reporting the model itself, not its solution
model.write("01_concrete_a.lp")

('Example1b.lp', 2233384673912)

***
### <span style="color:blue">Task</span>
Open the file "01_concrete_a.lp" with a text editor. Can you recognize the variables and constraints?
***

In [5]:
# Try this now
model.write("01_concrete_b.lp", io_options={'symbolic_solver_labels': True})

('Example1b.lp', 2233384675816)

Now let's solve the model!

In [6]:
opt = pyo.SolverFactory('glpk') # glpk: GNU Linear Programming Kit
results = opt.solve(model)
# First way of reporting the solution
results

{'Problem': [{'Name': 'unknown', 'Lower bound': 15750.0, 'Upper bound': 15750.0, 'Number of objectives': 1, 'Number of constraints': 16, 'Number of variables': 11, 'Number of nonzeros': 21, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.03581404685974121}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
print(results.Problem)


- Name: unknown
  Lower bound: 15750.0
  Upper bound: 15750.0
  Number of objectives: 1
  Number of constraints: 16
  Number of variables: 11
  Number of nonzeros: 21
  Sense: minimize



> For more on solver status and termination conditions:
http://www.pyomo.org/blog/2015/1/8/accessing-solver

In [13]:
model.display()

Model Example1

  Variables:
    s : Size=10, Index=s_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (2, 1) :     0 :  70.0 :  None : False : False : NonNegativeReals
        (2, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (3, 1) :     0 :  90.0 :  None : False : False : NonNegativeReals
        (3, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (4, 1) :     0 :  50.0 :  None : False : False : NonNegativeReals
        (4, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (5, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (5, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 15750.0

  Constraints:
 

***
### <span style="color:blue">Task</span>
1. Try to comment one or multiple constraints. What happens?
2. Try to maximize instead of minimizing the costs. (Tip: add the option 'sense=pyo.maximize' into the objective function)
3. How easy is it to add another power plant? Another time step?
***

## Formulation as a pyomo AbstractModel

One way to add flexibility is to write the problem abstractly. For example, the following equations represent a linear program (LP) to find optimal values for the vector $x$ (in our case, the hourly supply from the power plants) with parameters $c_j$ (costs), $a_{i,j}$ and $b_i$ (constraints):

$$ \begin{array}{lll} \min & \sum_{j=1}^n c_j x_{j,t} & \\
s.t. & \sum_{j=1}^n a_{i,j} x_{j,t} \geq b_{i,t} & \forall i = 1 \ldots m\\ & x_{j,t} \geq 0 & \forall j = 1 \ldots n
\end{array} $$ 

For that, there is the pyomo class AbstractModel:

In [1]:
from pyomo.environ import *

model = AbstractModel()

# Sets
model.I = Set() # we could define the dimensions, or let pyomo determine them from the data
model.J = Set()
model.T = Set()

# Parameters
model.a = Param(model.I, model.J)
model.b = Param(model.I, model.T)
model.c = Param(model.J)

# Variables
model.x = Var(model.J, model.T, domain=NonNegativeReals) # the variable is indexed by the set J

# Objective function
def obj_expression(model):
    sigma = 0
    for t in model.T:
        for j in model.J:
            sigma = sigma + model.c[j] * model.x[(j, t)]
    return sigma

model.OBJ = Objective(rule=obj_expression)

# Constraints
def ax_constraint_rule(model, i, t):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.x[j, t] for j in model.J) >= model.b[i, t]

model.AxbConstraint = Constraint(model.I, model.T, rule=ax_constraint_rule) # this creates one constraint for each member of the set model.I

***
### <span style="color:blue">Task</span>
With pen and paper, determine the parameters a, b, and c to replicate the concrete model.
***

By running the code, we create an abstract model. Now we need to create an instance of it:

In [2]:
# We can create an instance without filling it with data
instance = model.create_instance()
instance.pprint()

7 Set Declarations
    AxbConstraint_index : Dim=0, Dimen=2, Size=0, Domain=None, Ordered=False, Bounds=None
        Virtual
    I : Dim=0, Dimen=1, Size=0, Domain=None, Ordered=False, Bounds=None
        []
    J : Dim=0, Dimen=1, Size=0, Domain=None, Ordered=False, Bounds=None
        []
    T : Dim=0, Dimen=1, Size=0, Domain=None, Ordered=False, Bounds=None
        []
    a_index : Dim=0, Dimen=2, Size=0, Domain=None, Ordered=False, Bounds=None
        Virtual
    b_index : Dim=0, Dimen=2, Size=0, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index : Dim=0, Dimen=2, Size=0, Domain=None, Ordered=False, Bounds=None
        Virtual

3 Param Declarations
    a : Size=0, Index=a_index, Domain=Any, Default=None, Mutable=False
        Key : Value
    b : Size=0, Index=b_index, Domain=Any, Default=None, Mutable=False
        Key : Value
    c : Size=0, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value

1 Var Declarations
    x : Size=0, Index=x_index
     

In [3]:
# We can load data from a file (written in AMPL format)
data = DataPortal()
data.load(filename='01_abstract.dat')
# You can view the defined sets and parameters
list(data.keys())

['I', 'J', 'T', 'c', 'a', 'b']

***
### <span style="color:blue">Task</span>
Compare your results for the parameters a, b, and c with the used values in _01_abstract.dat_.
***

In [4]:
# We can create an instance that is filled with input data
instance = model.create_instance(data)
instance.pprint()

7 Set Declarations
    AxbConstraint_index : Dim=0, Dimen=2, Size=15, Domain=None, Ordered=False, Bounds=None
        Virtual
    I : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['BiomassCap', 'Dem', 'GasCap']
    J : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Biomass', 'Gas']
    T : Dim=0, Dimen=1, Size=5, Domain=None, Ordered=False, Bounds=None
        ['t1', 't2', 't3', 't4', 't5']
    a_index : Dim=0, Dimen=2, Size=6, Domain=None, Ordered=False, Bounds=None
        Virtual
    b_index : Dim=0, Dimen=2, Size=15, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index : Dim=0, Dimen=2, Size=10, Domain=None, Ordered=False, Bounds=None
        Virtual

3 Param Declarations
    a : Size=6, Index=a_index, Domain=Any, Default=None, Mutable=False
        Key                       : Value
        ('BiomassCap', 'Biomass') :    -1
            ('BiomassCap', 'Gas') :     0
               ('Dem', 'Biomass') :     1
       

In [9]:
opt = SolverFactory('glpk')
status = opt.solve(instance)
status

{'Problem': [{'Name': 'unknown', 'Lower bound': 15750.0, 'Upper bound': 15750.0, 'Number of objectives': 1, 'Number of constraints': 16, 'Number of variables': 11, 'Number of nonzeros': 21, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.014879226684570312}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
instance.display()

Model unknown

  Variables:
    x : Size=10, Index=x_index
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
        ('Biomass', 't1') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('Biomass', 't2') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('Biomass', 't3') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('Biomass', 't4') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('Biomass', 't5') :     0 :  30.0 :  None : False : False : NonNegativeReals
            ('Gas', 't1') :     0 :  30.0 :  None : False : False : NonNegativeReals
            ('Gas', 't2') :     0 :  70.0 :  None : False : False : NonNegativeReals
            ('Gas', 't3') :     0 :  90.0 :  None : False : False : NonNegativeReals
            ('Gas', 't4') :     0 :  50.0 :  None : False : False : NonNegativeReals
            ('Gas', 't5') :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
 

In [11]:
# Another way of reporting the results
instance.solutions.store_to(status)
status.write(filename='01_abstract_results.json', format='json')

***
### <span style="color:blue">Task</span>
1. Set the demand in the last time step to 140. What happens?
2. Add a third technology, PV, with zero running costs and with varying upper bounds for every time step.
3. How easy is it to add another power plant? Another time step?
***

## Reporting into pandas DataFrame objects

pandas is a package that allows you to organize your data in multidimensional "tables", so-called DataFrame objects. It is useful if you want to export your results into csv or Microsoft Excel format.

In [28]:
# load the library first
import pandas as pd

# we will create a dictionary from the model instance variable x
# the keys of the dictionary are the indices, the values are the x values
supply_data = {(j, t): value(x) for (j, t), x in instance.x.items()}

# create a DataFrame object from that dictionary
df_supply = pd.DataFrame.from_dict(supply_data, orient="index", columns=["P_out [MW]"])
df_supply

,P_out [MW]
"(Gas, t1)",30.0
"(Gas, t2)",70.0
"(Gas, t3)",90.0
"(Gas, t4)",50.0
"(Gas, t5)",0.0
"(Biomass, t1)",30.0
"(Biomass, t2)",30.0
"(Biomass, t3)",30.0
"(Biomass, t4)",30.0
"(Biomass, t5)",30.0


In [29]:
# Let's make the index look better - through a multiindex
df_supply.index = pd.MultiIndex.from_tuples(df_supply.index, names=('Technology', 't'))
# Get rid of the letter t in the timesteps
df_supply.index = df_supply.index.set_levels(df_supply.index.levels[1].str.replace('t', ''), level=1)
# S
df_supply

P_out [MW]
Technology timestep            
Gas        t1              30.0
           t2              70.0
           t3              90.0
           t4              50.0
           t5               0.0
Biomass    t1              30.0
           t2              30.0
           t3              30.0
           t4              30.0
           t5              30.0

In [35]:
df_supply.index = df_supply.index.set_levels(df_supply.index.levels[1].str.replace('t', ''), level=1)
df_supply

P_out [MW]
Technology timestep            
Gas        1               30.0
           2               70.0
           3               90.0
           4               50.0
           5                0.0
Biomass    1               30.0
           2               30.0
           3               30.0
           4               30.0
           5               30.0